In [1]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import numpy as np
import json

In [2]:
df = pd.read_json('raw_data.json', lines=True)
df.head()

,id,name,rating,ratings,released,metacritic,ratings_count,genres
0,3498,Grand Theft Auto V,4.48,"[{'id': 5, 'title': 'exceptional', 'count': 16...",2013-09-17,96.0,2758,"[Action, Shooter]"
1,4200,Portal 2,4.61,"[{'id': 5, 'title': 'exceptional', 'count': 16...",2011-04-19,95.0,2405,"[Shooter, Puzzle]"
2,3328,The Witcher 3: Wild Hunt,4.68,"[{'id': 5, 'title': 'exceptional', 'count': 20...",2015-05-18,93.0,2552,[RPG]
3,5679,The Elder Scrolls V: Skyrim,4.39,"[{'id': 5, 'title': 'exceptional', 'count': 10...",2011-11-11,NaN,2015,"[Action, RPG]"
4,12020,Left 4 Dead 2,4.08,"[{'id': 4, 'title': 'recommended', 'count': 76...",2009-11-17,89.0,1431,"[Action, Shooter]"


In [3]:
print(f'{df.describe()}\n')
print(f'{df.info()}\n')
print(f"released NA {df['released'].isna().sum()}\n")
print(f"metacritic NA {df['metacritic'].isna().sum()}\n")

                  id        rating   metacritic  ratings_count
count   11298.000000  11298.000000  2271.000000   11298.000000
mean    34967.389715      2.717164    73.616028      54.676226
std     57025.162227      1.504177    10.458881     144.520690
min         2.000000      0.000000    24.000000       4.000000
25%     10665.500000      2.000000    68.000000       6.000000
50%     19600.500000      3.250000    75.000000      13.000000
75%     38375.500000      3.870000    81.000000      36.000000
max    394092.000000      5.000000    97.000000    2758.000000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11298 entries, 0 to 11297
Data columns (total 8 columns):
id               11298 non-null int64
name             11298 non-null object
rating           11298 non-null float64
ratings          11298 non-null object
released         11088 non-null object
metacritic       2271 non-null float64
ratings_count    11298 non-null int64
genres           11298 non-null object
dtypes: float

In [4]:
df[df.released.isna()].sort_values(by='ratings_count', ascending=False).head(23)

,id,name,rating,ratings,released,metacritic,ratings_count,genres
226,5687,The Playroom,2.21,"[{'id': 1, 'title': 'skip', 'count': 210, 'per...",None,NaN,441,[]
548,42309,Dead Island: Epidemic,2.20,"[{'id': 1, 'title': 'skip', 'count': 63, 'perc...",None,NaN,136,[Arcade]
794,42391,Nosgoth,2.54,"[{'id': 1, 'title': 'skip', 'count': 40, 'perc...",None,NaN,97,"[Action, Shooter]"
1491,10096,3DMark,3.43,"[{'id': 4, 'title': 'recommended', 'count': 44...",None,NaN,79,[]
1247,23576,GOD EATER RESURRECTION,3.20,"[{'id': 3, 'title': 'meh', 'count': 28, 'perce...",None,NaN,71,[Action]
2516,9551,Deep Rock Galactic,4.14,"[{'id': 4, 'title': 'recommended', 'count': 40...",None,NaN,61,"[Action, Indie]"
2104,10156,Aliens: Colonial Marines Collection,2.60,"[{'id': 1, 'title': 'skip', 'count': 21, 'perc...",None,45.0,54,[Action]
4212,5673,Terminator Salvation,2.70,"[{'id': 3, 'title': 'meh', 'count': 21, 'perce...",None,NaN,40,"[Action, Shooter]"
807,18785,Sam & Max 303: They Stole Max's Brain!,2.76,"[{'id': 1, 'title': 'skip', 'count': 13, 'perc...",None,NaN,38,[Adventure]
3325,11587,Kenshi,4.16,"[{'id': 5, 'title': 'exceptional', 'count': 16...",None,NaN,38,"[Action, RPG, Strategy, Simulation, Indie]"


# Cleaning the dataset
    1. Filling or removing missing values
    2. Save the new dataset to a new file

In [5]:
# check which game is missing release date and manually fill those with over 30 votes
def add_release_dates(df):
    '''
    This function fills specific observations with specific release dates.
    Input:
        df: Original raw data dataframe
    Output:
        Partially filled release dates in the dataframe
    '''
    # star wars: obi wan
    df.at[11169, 'released'] = '2001-12-19'
    # god eater resurrection
    df.at[1247, 'released'] = '2015-10-29' 
    # deep rock galactic
    df.at[2516, 'released'] = '2018-02-28'
    # aliens: Colonial Marines Collection
    df.at[2104, 'released'] = '2013-02-12'
    # terminator Salvation
    df.at[4212, 'released'] = '2009-05-01'
    # kenshi
    df.at[3325, 'released'] = '2018-12-06'
    # Sam & Max 303: They Stole Max's Brain!
    df.at[807, 'released'] = '2010-06-22'
    # Sam & Max 303: Beyond the Alley of the Dolls
    df.at[812, 'released'] = '2010-07-20'
    # Sam & Max 303: The Penal Zone
    df.at[814, 'released'] = '2010-04-02'
    # Sam & Max 303: The City That Dares Not Sleep
    df.at[811, 'released'] = '2010-08-30'
    # Sam & Max 303:  The Tomb of Sammun-Mak
    df.at[813, 'released'] = '2010-05-18'
    # Sword of the Stars: The Pit
    df.at[2396, 'released'] = '2013-02-21'
    # SpellForce 2 Anniversary Edition
    df.at[2399, 'released'] = '2017-04-12'
    # raft
    df.at[4281, 'released'] = '2018-05-23'
    # Unheard
    df.at[5799, 'released'] = '2019-03-29'
    # Hector: Episode 2
    df.at[1167, 'released'] = '2011-06-25'
    
    # most of the games with higher vote count that are missing release date are closed/suspended
    # no use recommending them.
    df.dropna(subset=['released'], inplace=True)
    return df

df = add_release_dates(df)

# drop games that do not contain genres
df = df[df['genres'] != '[]']

# asses the NAN value situation
print(f'{df.metacritic.isna().sum()}\n')
print(f'{df.info()}/n')

8839

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11104 entries, 0 to 11297
Data columns (total 8 columns):
id               11104 non-null int64
name             11104 non-null object
rating           11104 non-null float64
ratings          11104 non-null object
released         11104 non-null object
metacritic       2265 non-null float64
ratings_count    11104 non-null int64
genres           11104 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 780.8+ KB
None/n


# Filling the metacritic scores for as many games as possible
    1. Retrieve the names of all games without metacritic score
    2. Use selenium to scrape scores

In [6]:
no_metacritic = np.array(df['name'][df['metacritic'].isna()])

In [7]:
from selenium import webdriver
import requests

In [8]:
def to_json(dictionary, filename):
    '''
    Input:
        game: dictionary containing the data to write to the file
        filename: a string stating the name and path you wish to save the file at
    Returns:
        json file
    '''
    with open(f'{filename}.json', 'a') as file:
            file.write(f'{json.dumps(dictionary)}\n')
    return

In [ ]:
def scrape_metacritic(names, filename):
    '''
    Input:
        names: list of games to scrape their score from metacritic
        filename: a string for saving the json file
    Returns:
        json file containing a list of dictionarys with the name of the game and the 
    '''
    scores = []
    for name in tqdm(names):
        game = {}
        # open metacritic
        driver = webdriver.Chrome()
        driver.get("https://www.metacritic.com/game")
        driver.implicitly_wait(11)
        
        try:
            # find search bar to search the, enter the game name, and click search
            driver.find_element_by_id('primary_search_box').send_keys(f'{name}')
            driver.implicitly_wait(2)
            driver.find_element_by_id('primary_menu_item_enter_search').click()

            # filter out non games
            driver.implicitly_wait(5)
            parent = driver.find_element_by_class_name("filter_area")
            driver.implicitly_wait(2)
            element = parent.find_elements_by_class_name("title")[2].click()
            
            # good reviews
            if driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game positive"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game positive"][1])')[0].text
            
            # no review yet
            elif driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game tbd"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game tbd"][1])')[0].text
            
            # mixed reviews
            elif driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game mixed"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game mixed"][1])')[0].text
            
            # bad reviews
            elif driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game negative"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game negative"][1])')[0].text
            
            else:
                score = "Not Found"             
        except:
            score = "Selenium Fail" 
        
        game[name] = score
        to_json(game, filename)
        scores.append(game)    
        driver.close()
    
    driver.quit()
    return scores
 
metacritic_scores = scrape_metacritic(no_metacritic[700:800], 'metacritic_scores')

 43%|██████████████████████████████████▊                                              | 43/100 [19:50<22:54, 24.11s/it]

In [20]:
metacritic_scores

[{'PAC-MAN Championship Edition DX+': 'Not Found'},
 {'The LEGO Movie - Videogame': '80'},
 {'Camera Obscura': 'tbd'},
 {'Gears of War: Ultimate Edition': '82'},
 {'Creativerse': 'tbd'},
 {'Sid Meier’s Ace Patrol: Pacific Skies': 'Not Found'},
 {'Toki Tori 2+': '75'},
 {'Warcraft 3: The Frozen Throne': '88'},
 {'Forza Horizon': '91'},
 {'Sky Force Anniversary': '77'},
 {'ReCore': 'tbd'},
 {'Forza Motorsport 5': '79'},
 {'Wolfenstein 3D': '77'},
 {'Transmissions: Element 120': 'Selenium Fail'},
 {'Gauntlet': 'tbd'},
 {'Dead Nation': '77'},
 {'Castlevania: Lords of Shadow': '85'},
 {'Ultra Street Fighter IV': '82'},
 {'The Walking Dead: A New Frontier': '81'},
 {'Ticket to Ride': '91'},
 {'Skyforge': 'Selenium Fail'},
 {'Master Levels for Doom II': 'tbd'},
 {'Canyon Capers': 'tbd'},
 {'Tomb Raider V: Chronicles': 'Not Found'},
 {'Memories of a Vagabond': 'tbd'},
 {'The Adventures of Shuggy': '77'},
 {'Batman: Arkham Origins Blackgate - Deluxe Edition': 'tbd'},
 {'Endorlight': 'tbd'},
 {'

In [11]:
raw_scores_df = pd.read_json('metacritic_scores.json', lines=True)
def fill_metacritic(df, scores, column):
    
    # create a single column df with all the scraped data found on the diagonal
    scores_df.set_index([scores.columns], inplace=True)
    scores_df = pd.DataFrame(np.diag(scores_df), index=scores_df.index, columns=['metacritic'])
    
    # fill the missing results from the scraped scores by index
    df.set_index(column, inplace=True)
    df.metacritic = df['metacritic'].fillna(scores_df['metacritic'])
    return df

,The Elder Scrolls V: Skyrim,Half-Life 2: Lost Coast,Metro 2033,Warframe,God of War,Red Dead Redemption 2,Horizon Zero Dawn,Half-Life 2: Deathmatch,Batman: Arkham Knight,Uncharted 4: A Thief’s End,...,Titan Quest: Immortal Throne,SNOW,Project Zomboid,Tomb Raider IV: The Last Revelation,Jetpack Joyride,App Game Kit: Easy Game Development,Super Smash Bros. Ultimate,Return Of The Obra Dinn,Sid Meier’s Ace Patrol,Sound Shapes
0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,tbd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Selenium Fail,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Not Found,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Selenium Fail,NaN,NaN
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Found,NaN


In [ ]:
scores_df.reset_index()

In [ ]:
cleaned_df = df.copy(deep=True)
cleaned_df.to_csv('cleaner_raw_data.csv')

In [ ]:
def str_tolist(df, column):
    new_df = df.copy(deep=True)
    all_genres = []
    for i, element in enumerate(df[column]):
        if type(element) != 'list':
            # create one big string
            string = ' '.join(element)

            # remove non alphabet
            string = re.sub('\s+', '', string)
            string = re.sub('\W+', ' ', string)
            
            # create list 
            list_strings = string.split(' ')[1:-1] # (first and last elements in the list are white spaces)
            all_genres.extend(list_strings)
            
            # replace the single string with a list of strings
            new_df[column][i] = list_strings

    return new_df
test_df = str_tolist(cleaned_df, 'Genres')
test_df.head(50)

In [ ]:
uniques = pd.unique(cleaned_df['Genres'].values.ravel('K'))
uniques[0][0]

In [ ]:
def add_genre_columns(df, column):
    # convert the strings in genres column to list
    df, uniques = str_tolist(df, column)
    
    #create dataframe with the column names
    genres_df = pd.DataFrame(columns=uniques)
    
    # join the original df with the new df
    df = df.join(column_df, how='left')
    
    # add function that uses the list strings to fill the values
    return df, list_strings

def fill_genre_columns(df, uniques):
    df2 = df.copy(deep=True)
    zeros = np.zeros(len(uniques))
    
    all_dummies = []
    for row in tqdm(df2.itertuples()):
        i = 1
        uniques_dic = dict(zip(uniques, zeros))
        while i < len(uniques):
            if row in uniques:
                uniques_dic[row[i]] = 1
                i += 1

        all_dummies.append(uniques_dic)
    return pd.DataFrame(all_dummies, columns=uniques)
    
df, uniques = add_genre_columns(cleaned_df, 'Genres')

In [ ]:
test = cleaned_df.copy(deep=True)

In [ ]:
def populate_df(df):
    """ This function turns the characters dataframe into dummy varaibles dataframe"""
    uniques = pd.unique(df.values.ravel('K'))
    zeros = np.zeros(len(uniques))

    # main df protection
    df = df.copy(deep=True)

    all_dummies = []
    for row in df.itertuples():
        i = 1
        uniques_dic = dict(zip(uniques, zeros))
        while i < 6:
            uniques_dic[row[i]] = 1
            i += 1

        all_dummies.append(uniques_dic)
    return pd.DataFrame(all_dummies, columns=uniques)

In [ ]:
import re

df3 = cleaned_df['Genres'].str.replace(r'\']', '')
df3 = pd.DataFrame(df3.str.split().values.tolist())
df3[0] = df3[0].str.replace(r'[', '')
df3[0] = df3[0].str.replace(r'\'', '')
df3[0] = df3[0].str.replace(r'\,', '')

df3
# df2 = cleaned_df.copy(deep=True)
# zeros = np.zeros(len(uniques))

# if any("abc" in s for s in some_list):

test_df = df2.join(df3, how='left')

i = 0
for row in tqdm(test_df.itertuples()):
    print(f'{row}\n')
    uniques_dic = dict(zip(uniques, zeros))
    print(f'{uniques_dic}\n')
    while i < len(uniques):
            if any(row in unique for unique in uniques):
                uniques_dic[row[i]] = 1
                i += 1
                if i == 20:
                    break
    break

In [ ]:
def create_genre_list(series):
    genre_list = []
    for genres in series:
        genre_list.append(genres)
    return genre_list

a = create_genre_list(genres_series)
type(a[-1])

In [ ]:
b = 


In [ ]:
b